https://chatgpt.com/g/g-u1ZJsrG7f-brainstorming-six-thinking-hats/c/24256d6d-8d12-4405-82e2-f59b3e759348

In [1]:
import os
import json
from collections import defaultdict
import pickle
from pprint import pprint
import re
import random
from typing import List, Dict, Tuple, Any
from ast import literal_eval

import pandas as pd
import numpy as np
from tqdm.notebook import trange, tqdm
from datasets import load_dataset
from dotenv import load_dotenv, find_dotenv

from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate

from reduce_keys import reduce_keys

In [2]:
load_dotenv(find_dotenv())

token = os.getenv("HUGGINGFACE_TOKEN")

In [3]:
from Neo4jConnection import Neo4jConnection

graphdb = Neo4jConnection(uri="neo4j://localhost:7687", user="neo4j", password="password", db="demo")

In [4]:
def load_products(path):
    products_df = pd.read_csv(path)
    products_df["categories"] = products_df["categories"].apply(literal_eval)
    products_df["details"] = products_df["details"].apply(literal_eval)
    products_df["description"] = products_df["description"].apply(literal_eval)
    products_df["features"] = products_df["features"].apply(literal_eval)
    return products_df

In [28]:
products_df = load_products("data/products_0.001.csv")

## Preprocessing

In [ ]:
category = "Beauty_and_Personal_Care"
dataset_name = "McAuley-Lab/Amazon-Reviews-2023"
ds_type = ("raw_review_", "raw_meta_")

dataset = load_dataset(dataset_name, ds_type[1] + category, trust_remote_code=True, token=token, split="full")
df = dataset.to_pandas()
df.info()

In [ ]:
# Step 0: Drop columns that are not needed
df.drop(columns=["videos", "bought_together", "subtitle", "author"], inplace=True)

# Step 1: Replace "None" with np.nan in specific object columns except for features, images, description, categories, details
columns_to_check = ["main_category", "title", "price", "store", "parent_asin"]
df[columns_to_check] = df[columns_to_check].replace("None", np.nan)

# Step 2: Drop rows where any of the specified columns is null
# capture the digits in price and convert to float
df["price"] = df["price"].str.extract(r"(\d+\.\d+|\d+)").astype(float)
df.dropna(
    subset=["main_category", "title", "price", "images", "store", "parent_asin"],
    inplace=True,
)

# Remove "Beauty & Personal Care" from each array in 'categories'
df['categories'] = df['categories'].apply(lambda x: [cat for cat in x if cat != "Beauty & Personal Care"])

"""
Step 3: Drop rows based on any of the length conditions
- if len() of the values in images, categories, or details is 0
- if both description and features have length 0
"""
df = df[
    (df["images"].apply(len) > 0)
    & (df["details"].apply(len) > 0)
    & (df["categories"].apply(len) > 0)
    & ~((df["description"].apply(len) == 0) & (df["features"].apply(len) == 0))
]

df.reset_index(drop=True, inplace=True)
df.info()

## Create Explicit KG from Products

1. Main Category nodes (`main_category`). Linked with `Subcategory` or `Product` nodes.
2. Subcategory nodes (`categories`). `BELONGS_TO` `Category` and linked with `Product` nodes.
3. Product nodes (`title`, `average_rating`, `rating_number`, `features`, `description`, `price`, `images`, `store`, `id`). `BELONGS_TO` `Subcategory` or `Category`
4. Price Range ($5 ranges based on unique values in `price`)

In [ ]:
df.info()

In [ ]:
df['main_category'].value_counts().head(15)

In [18]:
df_relevant = df[
    df["main_category"].isin(
        [
            "All Beauty",
            "Health & Personal Care",
            "Amazon Home",
            "AMAZON FASHION",
            "Premium Beauty",
        ]
    )
]
final_sampled_df = df_relevant

In [ ]:
# Explode the categories column to get one row per category
df_exploded = df_relevant.explode("categories")

# Stratified sampling by category values
# First, get a unique set of categories per product and stratify by these unique values
sample_frac = 0.001
sampled_df = (
    df_exploded.groupby("categories")
    .sample(frac=sample_frac)
    .reset_index(drop=True)
)

# After stratified sampling, remove duplicates to get the original product_asin sample
sampled_products = sampled_df[["parent_asin"]].drop_duplicates()

# Join back with the original dataframe to get the full sampled products with their category arrays
final_sampled_df = df[df["parent_asin"].isin(sampled_products["parent_asin"])].reset_index(drop=True)

print("Sample Size:", final_sampled_df.shape[0] / df_relevant.shape[0] * 100)
final_sampled_df.info()
final_sampled_df['main_category'].value_counts().head(15)

In [ ]:
# Delete all Category nodes
graphdb.query("MATCH (c:Category) DETACH DELETE c")

# Get unique values from the main_category column
unique_categories = final_sampled_df['main_category'].unique()

# Loop through each unique category and create a Category node
queries = []
for category in unique_categories:
    # Cypher query to create a Category node
    query = f"""
    CREATE (c:Category {{name: '{category}'}})
    """
    # add to the batch of queries
    queries.append(query)
# Execute the queries
graphdb.bulk_query(queries)

unique_category_count = final_sampled_df["main_category"].nunique()
query = "MATCH (c:Category) RETURN COUNT(c) as category_count"
result, _ = graphdb.query(query)
# Extract the count from the result
category_node_count = result[0]["category_count"]

is_equal = unique_category_count == category_node_count
print(f"Number of Category nodes ({category_node_count}) {'==' if is_equal else '!='} unique categories in the DataFrame ({unique_category_count})")

In [ ]:
# Delete existing Subcategory nodes
graphdb.query("MATCH (sc:Subcategory) DETACH DELETE sc")

# Explode the categories column into individual rows
df_exploded = final_sampled_df.explode('categories')

# Get the unique values from the categories column
unique_subcategories = df_exploded['categories'].unique()

# Loop through each unique subcategory and create a Subcategory node
queries = []
for category in unique_subcategories:
    # escape single quotes in the category name
    category = category.replace("'", "\\'")
    # Cypher query to create a Category node
    query = f"""
    CREATE (sc:Subcategory {{name: '{category}'}})
    """
    # Add to the batch of queries
    queries.append(query)
# Execute the queries
graphdb.bulk_query(queries)

unique_subcategory_count = df_exploded["categories"].nunique()
query = "MATCH (sc:Subcategory) RETURN COUNT(sc) as subcategory_count"
result, _ = graphdb.query(query)
# Extract the count from the result
subcategory_node_count = result[0]["subcategory_count"]
# Check if the number of Category nodes matches the unique categories in the DataFrame
is_equal = unique_subcategory_count == subcategory_node_count
print(f"Number of Subcategory nodes ({subcategory_node_count}) {'==' if is_equal else '!='} unique categories in the DataFrame ({unique_subcategory_count})")

In [28]:
# Remove duplicate (main_category, categories) pairs
df_cat_subcat = df_exploded[['main_category', 'categories']].drop_duplicates()

# Create the queries
queries = []
for index, row in df_cat_subcat.iterrows():
    # Escape single quotes in both main_category and categories
    main_category = row['main_category'].replace("'", "\\'")
    subcategory = row['categories'].replace("'", "\\'")

    # Cypher query to create Subcategory and Category nodes and the BELONGS_TO relationship
    query = f"""
    MERGE (c:Category {{name: '{main_category}'}})
    MERGE (sc:Subcategory {{name: '{subcategory}'}})
    MERGE (sc)-[:BELONGS_TO]->(c)
    """
    # Add to the batch of queries
    queries.append(query)
# Execute the queries
graphdb.bulk_query(queries);

In [30]:
def simplify_images(images):
    images_simple = defaultdict(list)
    for key in images:
        for link in images[key]:
            if link and link.startswith("https://"):
                images_simple[key].append(link)
    return images_simple

In [ ]:
product_data = []
relationships = []

for index, row in final_sampled_df.iterrows():
    product_id = row["parent_asin"]
    title = row["title"].replace("'", "\\'")
    average_rating = row["average_rating"]
    rating_number = row["rating_number"]
    price = row["price"]
    images = json.dumps(simplify_images(row["images"]))
    store = row["store"].replace("'", "\\'")
    subcategories = row["categories"]
    subcategories = [sc.replace("'", "\\'") for sc in subcategories]

    # Collect product data
    product_data.append({
        'product_id': product_id,
        'title': title,
        'average_rating': average_rating,
        'rating_number': rating_number,
        'price': price,
        'images': images,
        'store': store
    })

    # Collect relationships
    for subcategory in subcategories:
        relationships.append({
            'product_id': product_id,
            'subcategory': subcategory
        })

# Cypher query using UNWIND for bulk creation
product_query = """
UNWIND $products AS product
CREATE (p:Product {product_id: product.product_id})
SET p.title = product.title,
    p.average_rating = product.average_rating,
    p.rating_number = product.rating_number,
    p.price = product.price,
    p.images = product.images,
    p.store = product.store
""".strip()

relationship_query = """
UNWIND $relationships AS rel
MATCH (p:Product {product_id: rel.product_id})
MATCH (sc:Subcategory {name: rel.subcategory})
MERGE (p)-[:BELONGS_TO]->(sc)
""".strip()

# Run the queries
batch_size = 1000

# Delete existing Review nodes
delete_query = """
MATCH (p:Product)
CALL (p) { 
DETACH DELETE p
} IN TRANSACTIONS OF 10000 ROWS;
"""
graphdb.query(delete_query)
# Create the Product nodes
for i in trange(0, len(product_data), batch_size):
    batch = product_data[i:i + batch_size]
    graphdb.query(product_query, products=batch)

graphdb.query("DROP INDEX product_range_productid IF EXISTS")
graphdb.query("DROP INDEX subcategory_range_name IF EXISTS")
graphdb.query("CREATE INDEX product_range_productid FOR (p:Product) ON (p.product_id)")
graphdb.query("CREATE INDEX subcategory_range_name FOR (sc:Subcategory) ON (sc.name)")

for i in trange(0, len(relationships), batch_size):
    batch = relationships[i:i + batch_size]
    graphdb.query(relationship_query, relationships=batch)

In [33]:
def get_price_range(price, change_point=550, percentage=10):
    if price <= change_point:
        # Use percentage-based interval
        lower_bound = price * (1 - percentage / 100)
        upper_bound = price * (1 + percentage / 100)
    else:
        # Use logarithmic-based interval
        log_base = 1.1  # You can adjust this base for more or less aggressive intervals
        lower_bound = price / log_base
        upper_bound = price * log_base
    
    return int(lower_bound), int(upper_bound)

In [ ]:
price_ranges = set()
price_relationships = []

# Step 1: Iterate over each row in the DataFrame
for index, row in final_sampled_df.iterrows():
    product_id = row["parent_asin"]
    price = row["price"]

    # Step 2: Create price ranges
    lower_bound, upper_bound = get_price_range(price, percentage=15)

    # Add the price range to the set
    price_ranges.add((lower_bound, upper_bound))

    # Step 3: Collect relationships between product and price range
    price_relationships.append(
        {
            "product_id": product_id,
            "lower_bound": lower_bound,
            "upper_bound": upper_bound,
        }
    )

# Step 4: Insert unique PriceRange nodes
price_range_query = """
UNWIND $price_ranges AS pr
CREATE (r:PriceRange {lower_limit: pr.lower_bound, upper_limit: pr.upper_bound})
"""

# Prepare the price_ranges data for Cypher
price_range_data = [{"lower_bound": pr[0], "upper_bound": pr[1]} for pr in price_ranges]

# Insert PriceRange nodes
graphdb.query("MATCH (r:PriceRange) DETACH DELETE r")
graphdb.query(price_range_query.strip(), price_ranges=price_range_data)

# Step 5: Create Cypher query to link Products to PriceRange nodes
price_relationship_query = """
UNWIND $price_relationships AS rel
MATCH (p:Product {product_id: rel.product_id})
MATCH (r:PriceRange {lower_limit: rel.lower_bound, upper_limit: rel.upper_bound})
MERGE (p)-[:AROUND_PRICE]->(r)
"""

# Insert AROUND_PRICE relationships
for i in trange(0, len(price_relationships), batch_size):
    batch = price_relationships[i : i + batch_size]
    graphdb.query(
        price_relationship_query.strip(), price_relationships=price_relationships
    )

In [35]:
final_sampled_df.to_csv(f"data/products_{sample_frac}.csv", index=False)

## Create Explicit KG from Reviews

- `Reviewer`: user_id
- `Review`: rating, title, text, helpful_vote
- `REVIEWS`: based on user_id
- `WROTE`: based on parent_asin

The purpose is just stylistic. That is, whenever a user clicks a product, they see
- The most helpful reviews (or most recent if we're out of helpful ones)
- What people are saying about this (LLM-extracted keywords & summaries)

In [ ]:
reviews_df = pd.read_json(f"data/reviews_0.001.json")
reviews_df.reset_index(inplace=True)
reviews_df.info()
reviews_df.head()

In [5]:
# # Sort the DataFrame by 'parent_asin', 'helpful_vote' (descending), and 'timestamp' (descending)
# reviews_df_sorted = reviews_df.sort_values(by=['parent_asin', 'helpful_vote', 'timestamp'], ascending=[True, False, False])

# # Group by 'parent_asin' and take the top 5 reviews from each group
# reviews_df_top5 = reviews_df_sorted.groupby('parent_asin').head(5).reset_index(drop=True)
# reviews_df_top5.info()

In [ ]:
# Get unique user_ids
unique_users = reviews_df["user_id"].unique()
unique_users = [{"user_id": user} for user in unique_users]
print("Number of unique users:", len(unique_users))
# Create Reviewer nodes
graphdb.query("MATCH (r:Reviewer) DETACH DELETE r")
cypher = """
UNWIND $users AS reviewer
CREATE (r:Reviewer {user_id: reviewer.user_id})
"""
batch_size = 1000
for i in trange(0, len(unique_users), batch_size):
    batch = unique_users[i:i + batch_size]
    graphdb.query(cypher, users=batch)

# Count number of reviewers in graph
query = "MATCH (r:Reviewer) RETURN COUNT(r) as reviewer_count"
result, _ = graphdb.query(query)
reviewer_count = result[0]["reviewer_count"]
print("Number of Reviewer nodes:", reviewer_count)

In [ ]:
reviews_data = []
review_product_rels = []
review_reviewer_rels = []

for index, row in tqdm(reviews_df.iterrows(), total=reviews_df.shape[0]):
    review_id = row["index"]
    rating = row["rating"]
    try:
        title = row["title"].replace("'", "\\'")
        text = row["text"].replace("'", "\\'")
        timestamp = row["timestamp"].strftime("%Y-%m-%d %H:%M:%S")
    except:
        continue
    product_id = row["parent_asin"]
    reviewer_id = row["user_id"]
    helpful_vote = row["helpful_vote"]

    # Collect review data
    reviews_data.append(
        {
            "review_id": review_id,
            "rating": rating,
            "title": title,
            "text": text,
            "helpful_vote": helpful_vote,
            "timestamp": timestamp,
        }
    )

    # Collect relationships
    review_product_rels.append({"review_id": review_id, "product_id": product_id})
    review_reviewer_rels.append({"review_id": review_id, "reviewer_id": reviewer_id})

In [ ]:
# Delete existing Review nodes
delete_query = """
MATCH (r:Review)
CALL (r) { 
DETACH DELETE r
} IN TRANSACTIONS OF 10000 ROWS;
"""
graphdb.query(delete_query)

# Create Review nodes
batch_size = 1000
review_query = """
UNWIND $reviews AS review
CREATE (r:Review {review_id: review.review_id, rating: review.rating, title: review.title, text: review.text, helpful_vote: review.helpful_vote, timestamp: review.timestamp})
"""
for i in trange(0, len(reviews_data), batch_size):
    batch = reviews_data[i : i + batch_size]
    graphdb.query(review_query, reviews=batch)

# Drop existing indexes
graphdb.query("DROP INDEX review_range_reviewid IF EXISTS")
graphdb.query("DROP INDEX reviewer_range_userid IF EXISTS")
# Create index on Review nodes
graphdb.query("CREATE INDEX review_range_reviewid FOR (r:Review) ON (r.review_id)")
graphdb.query("CREATE INDEX reviewer_range_userid FOR (r:Reviewer) ON (r.user_id)")

# Create edge relationships between Review and Product nodes
review_product_query = """
UNWIND $relationships AS rel
MATCH (r:Review) WHERE r.review_id = rel.review_id
MATCH (p:Product) WHERE p.product_id = rel.product_id
CREATE (r)-[:REVIEWS]->(p)
"""
for i in trange(0, len(review_product_rels), batch_size):
    batch = review_product_rels[i : i + batch_size]
    graphdb.query(review_product_query, relationships=batch)
# Create edge relationships between Review and Reviewer nodes
review_reviewer_query = """
UNWIND $relationships AS rel
MATCH (r:Review) WHERE r.review_id = rel.review_id
MATCH (rv:Reviewer) WHERE rv.user_id = rel.reviewer_id
CREATE (rv)-[:WROTE]->(r)
"""
for i in trange(0, len(review_reviewer_rels), batch_size):
    batch = review_reviewer_rels[i : i + batch_size]
    graphdb.query(review_reviewer_query, relationships=batch)

In [ ]:
# Queries from review to reviewer will not be needed
graphdb.query("DROP INDEX reviewer_range_userid IF EXISTS")

## Extract Information from Products via LLM

### Tasks

- Extract (via LLM) **SEO-optimized keywords** from title, features, and description. Create `Keyword` nodes. Create `HAS_KEYWORD` rel b/w `Keyword` and `Products`
- Create (via LLM) **summary** of product from all of its attributes using a template. Then, create **embedding** of this summary for vector search. Use it to find products that are **substitutes** or **complements** of a product. A product is a substitute if it belongs to the same (or similar) categories, otherwise it is a complement. Complement can also be determined (better) via `bought_together`, however this is usually NULL. Another (hacky) technique for complements is to use an LLM to suggest titles for complements ("complements", "frequently used with", or "often mentioned together"), adding these edges to the KG, and when searching for complements, use full text search to find products that match these titles.  
- Incorporate use case scenarios into the KG to provide more intuitive and relatable explanations for users. For example, connecting the query “insomnia” to common use cases like “taking an Epsom salt bath before bed” or “diffusing lavender oil at night” creates a more meaningful connection between the product and the user’s problem.

### SEO-optimized keyword extraction

Extract (via LLM) **SEO-optimized keywords** from title, features, and description. Create `Keyword` nodes. Create `HAS_KEYWORD` rel b/w `Keyword` and `Products`

In [201]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from typing import List

from ast import literal_eval

In [ ]:
products_df = pd.read_csv("data/products_0.001.csv")
products_df.info()

In [ ]:
examples = [
    {
        "product_title": "Hydrating Facial Serum",
        "product_description": "A lightweight serum that deeply hydrates the skin, reducing the appearance of fine lines and wrinkles.",
        "product_features": "Hydrating, anti-aging, lightweight, non-greasy, fast-absorbing",
        "extractive_keywords": [
            "hydrating facial serum",
            "anti-aging serum",
            "lightweight serum"
        ],
        "abstractive_keywords": [
            "deeply hydrating skincare",
            "fine line reducer",
            "non-greasy facial treatment",
            "fast-absorbing serum"
        ]
    },
    {
        "product_title": "Organic Shea Butter Body Lotion",
        "product_description": "Rich and nourishing body lotion made with organic shea butter to moisturize and soften the skin.",
        "product_features": "Organic, shea butter, moisturizing, rich texture, skin-softening",
        "extractive_keywords": [
            "organic shea butter lotion",
            "moisturizing body lotion"
        ],
        "abstractive_keywords": [
            "rich body moisturizer",
            "natural skin softener",
            "nourishing body cream"
        ]
    },
    {
        "product_title": "Vitamin C Brightening Moisturizer",
        "product_description": "A daily moisturizer infused with Vitamin C to brighten the complexion and protect against environmental damage.",
        "product_features": "Vitamin C, brightening, daily moisturizer, antioxidant, SPF protection",
        "extractive_keywords": [
            "Vitamin C moisturizer",
            "brightening facial cream",
            "daily antioxidant moisturizer",
            "SPF protected moisturizer"
        ],
        "abstractive_keywords": [
            "complexion brightener",
            "environmental protection moisturizer",
            "daily skin defender"
        ]
    },
    {
        "product_title": "Matte Finish Lipstick",
        "product_description": "Long-lasting lipstick with a matte finish, available in a variety of vibrant colors.",
        "product_features": "Matte finish, long-lasting, vibrant colors, non-drying, cruelty-free",
        "extractive_keywords": [
            "matte lipstick",
            "long-lasting lip color"
        ],
        "abstractive_keywords": [
            "vibrant lipstick shades",
            "non-drying makeup",
            "cruelty-free lip products",
            "bold color lipstick"
        ]
    },
    {
        "product_title": "Exfoliating Facial Scrub",
        "product_description": "A gentle exfoliating scrub that removes dead skin cells and unclogs pores, leaving the skin smooth and refreshed.",
        "product_features": "Exfoliating, gentle, unclogs pores, smooth skin, refreshing",
        "extractive_keywords": [
            "exfoliating facial scrub",
            "gentle skin exfoliant",
            "smooth skin facial scrub"
        ],
        "abstractive_keywords": [
            "pore unclogging scrub",
            "refreshing exfoliator"
        ]
    },
    {
        "product_title": "Anti-Pollution Facial Mist",
        "product_description": "A refreshing facial mist that protects skin from urban pollution and hydrates throughout the day.",
        "product_features": "Anti-pollution, hydrating, refreshing, lightweight, convenient spray bottle",
        "extractive_keywords": [
            "anti-pollution facial mist",
            "hydrating facial mist"
        ],
        "abstractive_keywords": [
            "urban pollution protection",
            "daily hydration spray",
            "lightweight skin mist",
            "refreshing skin mist"
        ]
    },
    {
        "product_title": "Organic Aloe Vera Gel",
        "product_description": "Pure organic aloe vera gel to soothe and moisturize irritated skin.",
        "product_features": "Organic, aloe vera, soothing, moisturizing, natural",
        "extractive_keywords": [
            "organic aloe vera gel",
            "soothing skin gel",
            "moisturizing aloe gel"
        ],
        "abstractive_keywords": [
            "natural skin soother",
            "irritated skin moisturizer",
            "pure aloe treatment"
        ]
    },
    {
        "product_title": "Sunscreen SPF 50",
        "product_description": "Broad-spectrum sunscreen with SPF 50 protection to shield skin from harmful UV rays.",
        "product_features": "Broad-spectrum, SPF 50, water-resistant, lightweight, non-greasy",
        "extractive_keywords": [
            "sunscreen SPF 50",
            "broad-spectrum sunscreen",
            "water-resistant sunscreen"
        ],
        "abstractive_keywords": [
            "UV protection cream",
            "lightweight sunblock",
            "non-greasy sun protection"
        ]
    },
    {
        "product_title": "Retinol Night Cream",
        "product_description": "A potent night cream enriched with retinol to promote skin renewal and reduce wrinkles.",
        "product_features": "Retinol, night cream, skin renewal, wrinkle reduction, nourishing",
        "extractive_keywords": [
            "retinol night cream",
            "skin renewal cream"
        ],
        "abstractive_keywords": [
            "overnight wrinkle reducer",
            "nourishing night moisturizer",
            "retinol enriched cream",
            "anti-aging night treatment"
        ]
    },
    {
        "product_title": "Clarifying Acne Spot Treatment",
        "product_description": "Targeted spot treatment to reduce acne and prevent future breakouts.",
        "product_features": "Clarifying, acne treatment, spot treatment, quick-drying, non-irritating",
        "extractive_keywords": [
            "clarifying acne treatment",
            "spot treatment for acne",
            "acne spot treatment"
        ],
        "abstractive_keywords": [
            "breakout reducer",
            "quick-drying acne cream",
            "non-irritating blemish treatment"
        ]
    },
]

for i, example in enumerate(examples):
    examples[i]["product_features"] = example["product_features"].split(", ")

print(len(examples))

In [204]:
example_prompt = PromptTemplate(
    template="""Product:
- Title: {product_title}
- Description: {product_description}
- Features: {product_features}
Common Search Terms:
{{{{
"extractive_keywords": {extractive_keywords},
"abstractive_keywords": {abstractive_keywords}
}}}}
""".strip(),
    input_variables=[
        "product_title",
        "product_description",
        "product_features",
        "extractive_keywords",
        "abstractive_keywords",
    ],
    input_types={"product_title":  str, "product_description": str, "product_features": str, "extractive_keywords": List[str], "abstractive_keywords": List[str]},
)

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="""You are an expert in SEO-optimized keyword extraction. Your task is to extract common search terms that will make the product more discoverable in search engines.
Instructions:

Extract two types of keywords

1. extractive_keywords: List[str] = 'List of extractive SEO-optimized keywords (3-5). MUST be present in the product description.'
2. abstractive_keywords: List[str] = '''List of abstractive SEO-optimized keywords/phrases (3-5) AND
    MUST NOT be present in the product description BUT MUST BE related to the product AND
    MUST BE diverse AND NOT overlapping with the extractive keywords OR each other.'''

Guidelines:
1. **Relevance**: Extract keywords that are commonly used by potential customers searching for this product. Ensure the keywords are highly relevant to the product's features, use cases, and audience.
2. **Avoid Keyword Overlap**: If there are multiple synonyms, choose only the most relevant keyword. For example, if the product description contains both "cheap laptop" and "affordable laptop," choose one to avoid overlap and dilution.  
3. **Edge Weighting (Priority)**: Consider how specific and unique the keyword is to the product. More specific and descriptive terms (e.g., "lightweight laptop" for a travel laptop) should be prioritized over generic terms (e.g., "device").

Some examples are given below.
""",
    suffix="""
Your task: Extract the common search terms for the following product only.
Product:
- Title: {product_title}
- Description: {product_description}
- Features: {product_features}
Common Search Terms:
""",
    input_variables=["product_title", "product_description", "product_features"],
)

llm = ChatOllama(
    model="mistral-nemo",
    temperature=0.3, 
)
chain = few_shot_prompt | llm

In [15]:
# keyword_llm = llm.with_structured_output(KeywordExtraction)
# structured_chain = few_shot_prompt | keyword_llm

In [ ]:
# Step 7: Example Usage
input_data = {
    "product_title": "Hydrating Overnight Mask",
    "product_description": "An intensive overnight mask that restores moisture and revitalizes tired skin, leaving it soft and radiant by morning.",
    "product_features": "Hydrating, overnight treatment, revitalizing, moisture restoration, radiant skin"
}

# Invoke the chain with the input data
result = chain.invoke(input_data)
print(result.content)
# structured_result = structured_chain.invoke(input_data)

# print(structured_result.extractive_keywords)
# print(structured_result.abstractive_keywords)

In [ ]:
keywords_filename = "keywords.pkl"

if os.path.exists(keywords_filename):
    with open(keywords_filename, "rb") as f:
        keywords = pickle.load(f)
else:
    keywords = []

errors = []

for index, row in tqdm(products_df.iterrows(), total=products_df.shape[0]):
    if index < len(keywords):
        continue

    product_title = row["title"]
    product_description = row["description"]
    product_features = row["features"]

    if not product_description:
        product_description = "No description available."        
    
    inputs = {
        "product_title": product_title,
        "product_description": product_description[:5000],
        "product_features": product_features
    }
    try:
        chain_output = chain.invoke(inputs)
        # regex pattern to find dict
        dict_pattern = re.compile(r"\{[^\}]+\}")
        # call the LLM to format the output in the correct format
        prompt = f"""
Please format the below AI response into the following format:
{{
    "extractive_keywords": ["keyword1", "keyword2", "keyword3"],
    "abstractive_keywords": ["keyword4", "keyword5", "keyword6"]
}}
If there is more than one dictionary, just keep the first one.

{chain_output.content}
""".strip()
        llm_output = llm.invoke(prompt)
        parsed_output = dict_pattern.search(llm_output.content)
        if parsed_output is None:
            raise ValueError(f"Failed to parse the output for product {index}")
        # remove 's from the output to prevent issues with literal_eval
        content = parsed_output.group().replace("'s ", "s ")
        parsed_output = literal_eval(content)
        extractive_keywords = parsed_output['extractive_keywords']
        abstractive_keywords = parsed_output['abstractive_keywords']
        keywords.append({"extractive": extractive_keywords, "abstractive": abstractive_keywords})
        with open(keywords_filename, "wb") as f:
            pickle.dump(keywords, f)
    except Exception as e:
        print(f"Error processing product {index}: {e}")
        errors.append(index)
        keywords.append(None)        

In [ ]:
with open("errors_keywords.txt") as f:
    errors = f.read().splitlines()
    errors = [int(e) for e in errors]

for index in tqdm(errors):

    row = products_df.loc[index]
    product_title = row["title"]
    product_description = row["description"]
    product_features = row["features"]

    if not product_description:
        product_description = "No description available."

    inputs = {
        "product_title": product_title,
        "product_description": product_description[:5000],
        "product_features": product_features,
    }
    chain_output = chain.invoke(inputs)
    # regex pattern to find dict
    dict_pattern = re.compile(r"\{[^\}]+\}")
    # call the LLM to format the output in the correct format
    prompt = f"""
Please format the below AI response into the following format:
{{
"extractive_keywords": ["keyword1", "keyword2", "keyword3"],
"abstractive_keywords": ["keyword4", "keyword5", "keyword6"]
}}

If there is more than one dictionary, just keep the first one.

{chain_output.content}
""".strip()
    llm_output = llm.invoke(prompt)
    parsed_output = dict_pattern.search(llm_output.content)
    if parsed_output is None:
        raise ValueError(f"Failed to parse the output for product {index}")
    # remove 's from the output to prevent issues with literal_eval
    content = parsed_output.group().replace("'s ", "s ")
    parsed_output = literal_eval(content)
    extractive_keywords = parsed_output["extractive_keywords"]
    abstractive_keywords = parsed_output["abstractive_keywords"]
    keywords[index] = {
        "extractive": extractive_keywords,
        "abstractive": abstractive_keywords,
    }

In [208]:
with open(keywords_filename, "wb") as f:
    pickle.dump(keywords, f)

In [275]:
with open(keywords_filename, "rb") as f:
    keywords = pickle.load(f)

In [ ]:
bad_entries = []
for i, entry in enumerate(keywords):
    if entry is None:
        bad_entries.append(i)
print(len(bad_entries))

In [ ]:
stop = False
bad_entries = []
for i, entry in enumerate(keywords):
    for key in entry:
        for keyword in entry[key]:
            if not isinstance(keyword, str) or len(keyword) < 2:
                bad_entries.append(i)
                stop = True
                break
        if stop:
            break
print(bad_entries)

In [ ]:
stop = False
bad_entries = []
for i, entry in enumerate(keywords):
    for key in entry:
        if type(entry[key]) != list:
                bad_entries.append(i)
                break
print(bad_entries)

In [215]:
for bad_index in bad_entries:
    extractive_dict = keywords[bad_index]["extractive"]
    values = list(extractive_dict.values())
    # flatten the list
    values = [item for sublist in values for item in sublist]
    keywords[bad_index]["extractive"] = values

    abstractive_dict = keywords[bad_index]["abstractive"]
    values = list(abstractive_dict.values())
    # flatten the list
    values = [item for sublist in values for item in sublist]
    keywords[bad_index]["abstractive"] = values

In [ ]:
extractive_kw_lengths = []
abstractive_kw_lengths = []

for entry in keywords:
    extractive_kw_lengths.append(len(entry["extractive"]))
    abstractive_kw_lengths.append(len(entry["abstractive"]))

import seaborn as sns

# visualize distribution of lengths using boxplot
import matplotlib.pyplot as plt

# Create a figure and axis
fig, ax = plt.subplots(1, 2, figsize=(12, 6))

# Plot the extractive keyword lengths
sns.boxplot(extractive_kw_lengths, ax=ax[0])
ax[0].set_title('Distribution of Extractive Keyword Lengths')
ax[0].set_xlabel('Length')
ax[0].set_ylabel('Frequency')

# Plot the abstractive keyword lengths
sns.boxplot(abstractive_kw_lengths, ax=ax[1])
ax[1].set_title('Distribution of Abstractive Keyword Lengths')
ax[1].set_xlabel('Length')
ax[1].set_ylabel('Frequency')

# Set the main title for the figure
fig.suptitle('Keyword Length Distributions')

# Show the plot
plt.tight_layout()
plt.show()

In [39]:
with open("keywords.pkl", "rb") as f:
    keywords = pickle.load(f)

In [40]:
len(keywords)

958

In [41]:
keywords_flat = []
for index, entry in enumerate(keywords):
    values = list(entry.values())
    values_flat = values[0] + values[1]
    for value in values_flat:
        keywords_flat.append({"keyword": value, "product_index": index})

In [42]:
print(len({entry["keyword"] for entry in keywords_flat}))
keywords_reduced, unique_keywords = reduce_keys(keywords_flat, "keyword", similarity_threshold=90)
print(len(unique_keywords))

5488
5339


In [43]:
# Delete existing Keyword nodes
delete_query = """
MATCH (k:Keyword)
CALL (k) {
DETACH DELETE k
} IN TRANSACTIONS OF 10000 ROWS;
"""
graphdb.query(delete_query)

# Create Keyword nodes
keyword_query = """
UNWIND $keywords AS keyword
CREATE (k:Keyword {name: keyword})
"""
batch_size = 1000
unique_keywords = list(unique_keywords)
for i in trange(0, len(unique_keywords), batch_size):
    batch = unique_keywords[i:i + batch_size]
    graphdb.query(keyword_query, keywords=batch)

# Create HAS_KEYWORD relationship between Product (product_df[i]) and Keyword (new_keywords[i])
product_keyword_rels = [{"product_id": products_df.loc[entry["product_index"], "parent_asin"], "keyword": entry["keyword"]} for entry in keywords_reduced]

product_keyword_query = """
UNWIND $relationships AS rel
MATCH (p:Product {product_id: rel.product_id})
MATCH (k:Keyword {name: rel.keyword})
CREATE (p)-[:HAS_KEYWORD]->(k)
"""
for i in trange(0, len(product_keyword_rels), batch_size):
    batch = product_keyword_rels[i:i + batch_size]
    graphdb.query(product_keyword_query, relationships=batch)

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

### Key Attribute Extraction

With generative AI, we may significantly enhance the quality of catalog data. Here is an example of using LLM to tag products with specific features. In a typical catalog, a product is listed with a name and description. For example:

**Product name**:
*Swiffer Antibacterial Cleaner, Febreze Citrus & Light Scent, Refill*

**Product description**:
*Kills 99.9% of bacteria (Kills 99.9% of staphylococcus aureus and enterobacter aerogenes). Helps eliminate odors in the air with a fresh scent. Great for vinyl, glazed ceramic, sealed marble, laminate, and finished wood floors. Do not use on unfinished, oiled, or waxed wooden boards, non-sealed tiles, or carpeted floors because they may be water sensitive. Good Housekeeping: Since 1909. Limited warranty to consumers. Replacement or refund if defective. Contains no phosphate, chlorine bleach or ammonia. Questions? 1–800–742–9220. www.swiffer.com. Bottle fits all Swiffer Wet Jet devices”*

By utilizing an LLM, we can extract and tag key attributes from this information, improving the product’s visibility in search results and its alignment with user needs. The LLM might generate output like this:

    {
    "Antibacterial": true,
    "Scent": "Febreze Citrus & Light",
    "Suitable Floor Types": ["Vinyl", "Glazed Ceramic", "Sealed Marble", "Laminate", "Finished Wood"],
    "Not Suitable Floor Types": ["Unfinished, Oiled, or Waxed Wooden Boards", "Non-Sealed Tiles", "Carpeted Floors"],
    "Phosphate Free": true,
    "Chlorine Bleach Free": true,
    "Ammonia Free": true,
    "Brand": "Swiffer",
    "Compatibility": ["Fits all Swiffer Wet Jet devices"],
    "Product Type": "Floor Cleaner",
    "Package Type": "Refill"
    }

This detailed and structured output not only aids in better product discovery but also ensures a more precise match with customer queries and preferences.

In [59]:
from langchain_core.messages import AIMessage

prompt_text = """You are an advanced language model tasked with enhancing product catalogs by extracting key attributes from product titles and descriptions. You will be provided with an existing dictionary of attributes, which may sometimes be empty. Your goal is to analyze the product title and description to identify and add any missing attributes, or modify existing, ensuring the dictionary is accurate and comprehensive.

Here is an example of how to process a product's information:

**Product Title**: Swiffer Antibacterial Cleaner, Febreze Citrus & Light Scent, Refill  
**Product Description**: Kills 99.9% of bacteria (Kills 99.9% of staphylococcus aureus and enterobacter aerogenes). Helps eliminate odors in the air with a fresh scent. Great for vinyl, glazed ceramic, sealed marble, laminate, and finished wood floors. Do not use on unfinished, oiled, or waxed wooden boards, non-sealed tiles, or carpeted floors because they may be water sensitive. Good Housekeeping: Since 1909. Limited warranty to consumers. Replacement or refund if defective. Contains no phosphate, chlorine bleach or ammonia. Questions? 1–800–742–9220. www.swiffer.com. Bottle fits all Swiffer Wet Jet devices.

**Key Attributes**:
```json
{{
    "Antibacterial": true,
    "Scent": "Febreze Citrus & Light",
    "Suitable Floor Types": ["Vinyl", "Glazed Ceramic", "Sealed Marble", "Laminate", "Finished Wood"],
    "Not Suitable Floor Types": ["Unfinished, Oiled, or Waxed Wooden Boards", "Non-Sealed Tiles", "Carpeted Floors"],
    "Phosphate Free": true,
    "Chlorine Bleach Free": true,
    "Ammonia Free": true,
    "Brand": "Swiffer",
    "Compatibility": "Fits all Swiffer Wet Jet devices",
    "Product Type": "Floor Cleaner",
    "Package Type": "Refill"
}}
```

Using this as a guide, analyze the product title and description provided below to extract relevant attributes and populate the existing dictionary accordingly. Add new attributes if necessary, but ensure that:

- All values are either **primitive types** (strings, booleans, numbers) or **lists**.
- Do not produce nested structures (like dictionaries or complex objects).
- If you find a delimiter (e.g., comma) in the text or a value in the existing attributes, reformat it as a list instead.
- Output the dictionary in JSON format with only primitive or list values. Do not include length 1 lists for single values. Use the primitives instead.
- Keep the keys in the dictionary consistent with the existing attributes or simplify them. If you find a new attribute, add it to the dictionary with the correct key name.

For example, this Key Attributes result is wrong
```json
{{
    "Age Range (Description)": "Adult", # key should be simplified to "Age Range"
    "Product": {{"Brand": "COVERGIRL", "Name": "Tone Rehab 2-in-1 Foundation"}} # dictionaries are not allowed
    "Item Form": "Liquid",
    "Package Dimensions": ["5.9", "2.3", "2 inches"],
    "Scent": "Banana",
    "Special Feature": ["Scented"], # value should be "Scented"
    "UPC": "678634485830",
    "Weight": ["4.8 Ounces"], # value should be "4.8 Ounces"
}}
```

This is correct
```json
{{
    "Age Range": "Adult",
    "Brand": "COVERGIRL",
    "Name": "Tone Rehab 2-in-1 Foundation",
    "Item Form": "Liquid",
    "Package Dimensions": ["5.9", "2.3", "2 inches"],
    "Scent": "Banana",
    "Special Feature": "Scented",
    "UPC": "678634485830",
    "Weight": "4.8 Ounces"
}}
```

**Product Title**: {product_title}  
**Product Description**: {product_description}  
**Existing Attributes**: {product_details}

Generate only the updated dictionary inside a json block as shown in the examples above.
"""
prompt_template = PromptTemplate(
    template=prompt_text,
    input_variables=["product_title", "product_description", "product_details"],
    input_types={
        "product_title": str,
        "product_description": str,
        "product_details": Dict[str, Any],
    },
)

llm = ChatOllama(
    model="mistral-nemo",
    temperature=0.3,
)

# capture everything between ```json and ```)
json_pattern = re.compile(r'(?<=```json)(.*?)(?=```)', re.DOTALL)

def parser(message: AIMessage) -> Dict[str, Any]:
    attributes = {}
    # find first match
    match = json_pattern.search(message.content)
    if match:
        attributes_str = match.group(0).strip()
        try:
            attributes = literal_eval(attributes_str)
        except ValueError:
            try:
                attributes = json.loads(attributes_str)
            except json.JSONDecodeError:
                pass
                # raise ValueError(f"Failed to parse the attributes dictionary {attributes_str}")
    return attributes

chain = prompt_template | llm | parser

In [68]:
from langchain.globals import set_debug


def run_example(example=None):
    set_debug(True)
    if example is not None: 
        example = products_df.sample()
    features = example["details"].values[0]
    features["store"] = example["store"].values[0]
    product_tile = example["title"].values[0]
    product_description = " ".join(example["description"].values[0])

    inputs = {
        "product_title": product_tile,
        "product_description": product_description,
        "product_details": features,
    }
    result = chain.invoke(inputs)
    set_debug(False)
    return result

# result = run_example()

In [ ]:
product_attributes = []
errors = []

output_filename = "product_attributes.json"

for index, row in tqdm(products_df.iterrows(), total=products_df.shape[0]):
    features = literal_eval(row["details"])
    features["store"] = row["store"]
    product_tile = row["title"]
    product_description = " ".join(row["description"])
    inputs = {
        "product_title": product_tile,
        "product_description": product_description,
        "product_details": features,
    }
    try:
        result = chain.invoke(inputs)
        product_attributes.append(result)
        with open(output_filename, "w") as f:
            json.dump(product_attributes, f)
    except Exception as e:
        print(f"Error processing product {index}: {e}")
        errors.append(index)
        product_attributes.append(None)

with open(f"errors_{output_filename[:-4]}.txt", "w") as f:
    for error in errors:
        f.write(f"{error}\n")

In [51]:
### resolve errors here
with open(f"errors_{output_filename[:-4]}.txt") as f:
    errors = f.read().splitlines()
    errors = [int(e) for e in errors]

with open(output_filename) as f:
    product_attributes = json.load(f)

In [100]:
def validate_output(output):
    for key in output:
        if type(output[key]) not in (str, bool, int, list):
            return False
        if type(output[key]) == list:
            for element in output[key]:
                if type(element) not in (str, bool, int):
                    return False
    return True

In [57]:
solved = []

In [ ]:
for index in tqdm(errors):
    if index in solved:
        continue
    
    row = products_df.loc[index]
    features = row["details"]
    features["store"] = row["store"]
    product_tile = row["title"]
    product_description = " ".join(row["description"])
    inputs = {
        "product_title": product_tile,
        "product_description": product_description,
        "product_details": features,
    }
    i = 0
    while i < 5:
        try:
            result = chain.invoke(inputs)
        except SyntaxError as e:
            print(f"Syntax error for product {index} on attempt {i + 1}")
            i += 1
            continue
        if result and validate_output(result):
            product_attributes[index] = result
            solved.append(index)
            print(f"Fixed error for product {index} on attempt {i + 1}")
            break
        i += 1
        print(f"Attempt {i} failed for product {index}")
if len(solved) != len(errors):
    print(f"Fixed {len(solved)} bad entries! {len(errors) - len(solved)} remaining...")

In [104]:
output_filename = "product_attributes.json"
with open(output_filename, "w") as f:
    json.dump(product_attributes, f)

In [105]:
for i, entry in enumerate(product_attributes):
    if type(entry) == list:
        for j, element in enumerate(entry):
            try:
                # try to convert to float if it's not a boolean
                if type(element) != bool:
                    element = float(element)
            except ValueError:
                # in case of string
                pass
            else:
                # if successful, replace the element with the float
                entry[j] = element

In [106]:
with open("product_attributes.json", "r") as f:
    product_attributes = json.load(f)

In [ ]:
attributes_set = set()
attributes_data = []
relationships = []

# Iterate over each row in the DataFrame
for index, row in products_df.iterrows():
    product_id = row["parent_asin"]
    product_details = product_attributes[index]

    for key, value in product_details.items():
        # Escape single quotes in the key
        key = key.replace("'", "\\'")

        if type(value) == list:
            for element in value:
                # Add attribute node data 
                attribute_tuple = (key, element)
                attribute = {
                    'name': key,
                    'value': element
                }
                if attribute_tuple not in attributes_set:
                    attributes_data.append(attribute)
                    attributes_set.add(attribute_tuple)

                # Collect relationships between product and attribute nodes
                relationships.append({
                    'product_id': product_id,
                    'attribute_name': key,
                    'attribute_value': element
                })
        else:
            # Add attribute node data 
            attribute_tuple = (key, value)
            attribute = {
                'name': key,
                'value': value
            }
            
            if attribute_tuple not in attributes_set:
                attributes_data.append(attribute)
                attributes_set.add(attribute_tuple)

            # Collect relationships between product and attribute nodes
            relationships.append({
                'product_id': product_id,
                'attribute_name': key,
                'attribute_value': value
            })

# Delete existing Attribute nodes
delete_query = """
MATCH (a:Attribute)
CALL (a) { 
DETACH DELETE a
} IN TRANSACTIONS OF 10000 ROWS;
"""
graphdb.query("MATCH (a:Attribute) DETACH DELETE a")
# Create Attribute nodes
attribute_query = """
UNWIND $attributes AS attr
CREATE (a:Attribute {name: attr.name, value: attr.value})
"""
# Insert attribute nodes
for i in trange(0, len(attributes_data), batch_size):
    batch = attributes_data[i:i + batch_size]
    graphdb.query(attribute_query.strip(), attributes=batch)

# Create indexes for the Attribute nodes
graphdb.query("DROP INDEX attribute_range_name IF EXISTS")
graphdb.query("CREATE INDEX attribute_range_name FOR (a:Attribute) ON (a.name)")
# Create HAS_ATTRIBUTE relationships
relationship_query = """
UNWIND $relationships AS rel
MATCH (p:Product {product_id: rel.product_id})
MATCH (a:Attribute {name: rel.attribute_name, value: rel.attribute_value})
MERGE (p)-[:HAS_ATTRIBUTE]->(a)
"""
# Create relationships between Product and Attribute nodes
for i in trange(0, len(relationships), batch_size):
    batch = relationships[i:i + batch_size]
    graphdb.query(relationship_query.strip(), relationships=batch)

### Product Summary

Create **summary** of product from all of its attributes using a template. Then, create **embedding** of this summary for vector search. Use it to find products that are **substitutes** or **complements** of a product. A product is a substitute if it belongs to the same (or similar) categories, otherwise it is a complement. Complement can also be determined (better) via `bought_together`, however this is usually NULL. Another (hacky) technique for complements is to use an LLM to suggest titles for complements ("complements", "frequently used with", or "often mentioned together"), adding these edges to the KG, and when searching for complements, use full text search to find products that match these titles.  

The generation of a product embedding requires us to take more information into consideration. Most current systems use only product names to produce the embedding, which fails to fully capture the product’s characteristics. It’s more effective to compile all the available data about a product and convert this complete information into text. This can be achieved by joining all related tables — including product, brand, category, taxonomy, and others — into a singular, “flattened” table.

The next step involves transforming the structured data in the “flattened” table into a textual format. The text is then converted into embeddings and stored in the vector database. Essentially, the LLM, combining the semantics represented by the table schema with the specific data values of each product, crafts a natural language description of the product. 

In [142]:
def generate_product_description(row):
    """
    Generate a product description using a template from a DataFrame row.

    Parameters:
    row (pd.Series): A row from the DataFrame containing product data.

    Returns:
    str: A product description generated according to Template 1.
    """

    # Extract mandatory fields with default fallbacks
    product_title = row.get("title", "N/A")
    main_category = row.get("main_category", "N/A")
    brand_store = row.get("store", "N/A")
    price = row.get("price", "N/A")
    average_rating = row.get("average_rating", "N/A")
    rating_number = row.get("rating_number", "N/A")

    # Parse 'categories' field
    categories = row.get("categories", [])
    if isinstance(categories, str):
        try:
            categories = literal_eval(categories)
        except (ValueError, SyntaxError):
            categories = [categories]
    categories_str = ", ".join(categories) if categories else "N/A"

    # Parse 'details' field
    details = row.get("details", {})
    if isinstance(details, str):
        try:
            # Clean up the string to make it valid JSON
            details = details.replace('""', '"').replace("'", '"')
            details = json.loads(details)
        except json.JSONDecodeError:
            details = {}
    # Extract optional fields
    material = details.get("Material", None)
    color = details.get("Color", None)
    product_dimensions = details.get("Package Dimensions") or details.get(
        "Product Dimensions"
    )
    weight = details.get("Item Weight")

    # Prepare 'product features' string
    features = row.get("features", [])
    if isinstance(features, str):
        try:
            features = literal_eval(features)
        except (ValueError, SyntaxError):
            features = [features]
    features = [f.strip("'\"") for f in features if f]
    key_features_str = "; ".join(features) if features else "N/A"

    # Collect additional details (excluding already used keys)
    used_keys = {
        "Material",
        "Color",
        "Package Dimensions",
        "Product Dimensions",
        "Item Weight",
        "Brand",
    }
    additional_details = [
        f"{key} - {value}" for key, value in details.items() if key not in used_keys
    ]
    additional_details_str = (
        ", ".join(additional_details) if additional_details else "N/A"
    )

    # Construct the description using Template 1
    description_parts = [
        f"{product_title} is a product under the {main_category} category.",
        f"Manufactured by {brand_store}, it is priced at ${price}.",
    ]

    # Include optional fields if available
    if material or color:
        material_color = " and ".join(filter(None, [material, color]))
        description_parts.append(f"The product features {material_color}.")
    else:
        description_parts.append("")

    if product_dimensions or weight:
        dimensions_weight = " and ".join(
            filter(
                None,
                [
                    (
                        f"dimensions of {product_dimensions}"
                        if product_dimensions
                        else None
                    ),
                    f"weighs {weight}" if weight else None,
                ],
            )
        )
        description_parts.append(f"It has {dimensions_weight}.")
    else:
        description_parts.append("")

    description_parts.append(f"Key features include {key_features_str}.")
    description_parts.append(
        f"It has an average rating of {average_rating} stars based on {rating_number} reviews."
    )
    description_parts.append(f"It falls under the subcategories: {categories_str}.")
    description_parts.append(f"Additional details: {additional_details_str}.")

    # Filter out any empty strings
    description_parts = [part for part in description_parts if part]

    # Join all parts into the final description
    description = " ".join(description_parts)

    return description

In [ ]:
row = products_df.sample(1).iloc[0]
description = generate_product_description(row)
print(len(description))
pprint(description)

In [253]:
from pydantic import BaseModel, Field


class ProductSummary(BaseModel):
    summary: str = Field(
        title="summary",
        description="A concise, holistic, and natural sounding summary of the product.",
        min_length=10
    )


prompt_text = """Rephrase the following product description to create a natural, human-friendly narrative while ensuring it provides detailed information for product research. 
The description should flow naturally, avoiding overly technical language or mechanical listing of features, but must still be comprehensive enough to allow for detailed product analysis.

Ensure the description includes the following:
- Product design, material, and functionality.
- Key features and how they benefit the user.
- Primary use cases and scenarios where the product excels.
- Compatibility with related products or items in the same category.

Guidelines:
- Keep the tone neutral and informative, focusing on providing clear, useful information that would help in comparing this product with others.
- Avoid using a salesy or promotional tone. 
- The description should feel natural and readable, but the completeness of information is the priority.
- Do not generate the result using markdown

Product Description: ```{description}```"""


prompt_template = PromptTemplate(
    template=prompt_text,
    input_variables=["description"],
    input_types={"description": str},
)

llm = ChatOllama(
    model="mistral-nemo",
    temperature=0.3,
)
summary_llm = llm.with_structured_output(ProductSummary, include_raw=True)
summary_chain = prompt_template | summary_llm

In [ ]:
from langchain.globals import set_debug

def run_example(example=None):
    set_debug(True)

    if example is None:
        example = products_df.sample().iloc[0]
        print(example.name)
    
    product_description = generate_product_description(example)
    result = summary_chain.invoke({"description": product_description})
    
    set_debug(False)
    return result

In [ ]:
result = run_example()

In [ ]:
print(result['raw'].content)

In [ ]:
print(result['parsed'])

In [230]:
def preprocess_string(input_str):
    """
    Preprocesses the input string by extracting and escaping the "summary" field.

    Steps:
    1. Check if the string starts with [TOOL_CALLS]. If not, return None.
    2. Remove the [TOOL_CALLS] prefix.
    3. Manually extract the "summary" value, accounting for unescaped quotes.
    4. Escape backslashes, double quotes, and newline characters within the summary.

    Args:
        input_str (str): The input string to preprocess.

    Returns:
        str or None: The escaped summary string if successful, otherwise None.
    """
    PREFIX = '[TOOL_CALLS]'

    # Step 1: Check for the prefix
    if not input_str.startswith(PREFIX):
        return None

    # Step 2: Remove the prefix
    json_str = input_str[len(PREFIX):]

    # Step 3: Manually extract the summary value
    summary_key = '"summary": "'
    start_idx = json_str.find(summary_key)
    if start_idx == -1:
        return None
    start_idx += len(summary_key)

    # Find the index of the closing quote before '}}]'
    end_marker = '"}}]'
    end_idx = json_str.rfind(end_marker)
    if end_idx == -1:
        return None

    # Extract the raw summary text
    summary_text = json_str[start_idx:end_idx]

    # Step 4: Escape necessary characters
    summary_text = summary_text.replace('\\', '\\\\')  # Escape backslashes first
    summary_text = summary_text.replace('"', '\\"')    # Escape double quotes
    summary_text = summary_text.replace('\n', '\\n')   # Escape newlines

    return summary_text

In [ ]:
summary = preprocess_string(result['raw'].content)
print(summary)

In [262]:
with open("data/summaries.json") as f:
    summaries = json.load(f)

In [ ]:
### error resolution...
with open("data/errors_summaries.txt") as f:
    errors = f.read().splitlines()
    errors = [int(e) for e in errors]
print(errors)

solved = []

In [ ]:
index = errors[2] # 0, 1, 3, 4
row = products_df.loc[index]
result = run_example(row)

In [ ]:
print(result['parsed'])

if result['parsed']:
    summaries.append({"index": index, "summary": result['parsed'].summary})

In [ ]:
raw_result = result['raw'].content
print(raw_result)

In [ ]:
raw_result = raw_result[raw_result.find("Product Summary:") + len("Product Summary:"):].strip()
print(raw_result)

In [296]:
raw_result_new = raw_result.replace("\n\n", "")

In [ ]:
print(raw_result_new)

In [ ]:
preprocessed_result = preprocess_string(raw_result)
print(preprocessed_result)

In [302]:
summaries.append({"index": index, "summary": "The ARTERO X-Tron Professional Hairstyling Clippers are designed for flexibility, with both cordless and plug-in use. They offer three adjustable speeds (5500, 6000, 6500 RPM) displayed on an LCD screen and come with comb attachments for trimming heights from 3mm to 12mm. A lithium battery provides up to 150 minutes of runtime, with a quick 90-minute charge. The clippers are lightweight and suitable for all hair types, making them ideal for professional use. Maintenance is simple with the included mineral oil and compatibility with ARTERO's Oil Fresh Spray for cleaning and disinfecting."})

In [303]:
with open("data/summaries.json", "w") as f:
    json.dump(summaries, f)

In [37]:
summaries = pd.read_json("data/summaries.json")
summaries

,index,summary
0,0,The Fresh Beards Classic Beard Butter is a met...
1,1,The DOORES Wire Hair Extensions offer a seamle...
2,2,Fingernail Friends Colorful Nail Stickers is a...
3,3,The MISUD Long Press on Nails is a high-qualit...
4,4,The Shikai Henna Gold Highlighting Shampoo is ...
...,...,...
953,156,Gold Bond's Overnight Deep Moisturizing Lotion...
954,161,The NOACIER Blemish Defense Night Face Serum i...
955,156,The Luinabio Chinese Classical Glaze Hair Stic...
956,681,The Brocato Supersilk Pure Indulgence Shampoo ...


In [40]:
product_summaries_df = pd.merge(products_df.reset_index(), summaries, on="index").drop(columns=["index"])
product_summaries_df.head()

,main_category,title,average_rating,rating_number,price,images,store,categories,details,parent_asin,features,description,summary
0,All Beauty,Fresh Beards Classic Beard Butter - Unscented ...,4.6,600,19.99,{'hi_res': array(['https://m.media-amazon.com/...,Fresh Beards,"[Shave & Hair Removal, Men's, Beard & Mustache...","{'Brand': 'Fresh Beards', 'Item Form': 'Butter...",B0BCXCYS2Q,[CLASSIC BEARD BUTTER: Our original unscented ...,[],The Fresh Beards Classic Beard Butter is a met...
1,All Beauty,DOORES Hair Extensions Wire Hair Extensions Ba...,4.3,2019,98.99,{'hi_res': array(['https://m.media-amazon.com/...,DOORES,"[Hair Care, Hair Extensions, Wigs & Accessorie...","{'Brand': 'DOORES', 'Color': '#(2/6)/2 Dark Br...",B08LKG6CWL,[1.【100% Human Hair Extensions - 9A Salon Qual...,[],The DOORES Wire Hair Extensions offer a seamle...
2,All Beauty,Fingernail Friends Colorful Nail Stickers Nail...,3.0,9,11.99,{'hi_res': array(['https://m.media-amazon.com/...,allydrew,"[Foot, Hand & Nail Care, Nail Art & Polish, Na...","{'Brand': 'Allydrew', 'Color': '2 Pack', 'Item...",B07PNRZLBH,[Includes: 50 Fairytale Nail Stickers & 50 Cut...,[Our Fingernail Friends Colorful Nail Stickers...,Fingernail Friends Colorful Nail Stickers is a...
3,All Beauty,"MISUD Long Press on Nails, Coffin Fake Nails, ...",3.8,372,7.66,{'hi_res': array(['https://m.media-amazon.com/...,MISUD,"[Foot, Hand & Nail Care, Nail Art & Polish, Fa...","{'Color': 'Blue', 'Size': 'Long, Coffin/Baller...",B08ZSB814R,[✨[HIGH QUALITY] -Our nails are made with high...,[PACKAGE CONTENTS:1 X 24pcs Fake Nails 1 X Dou...,The MISUD Long Press on Nails is a high-qualit...
4,All Beauty,"Shikai - Henna Gold Highlighting Shampoo, Brin...",4.4,110,36.00,{'hi_res': array(['https://m.media-amazon.com/...,ShiKai,"[Hair Care, Hair Coloring Products, Hennas]","{'Is Discontinued By Manufacturer': 'No', 'Pro...",B018RLF49U,[TURN ON THE LIGHTS TO YOUR HAIR’S SHIMMERING ...,"[In 1970, Dr. Dennis Sepp, an organic chemist,...",The Shikai Henna Gold Highlighting Shampoo is ...


In [41]:
pid_summary = []

for _, row in product_summaries_df.iterrows():
    product_id = row["parent_asin"]
    summary = row["summary"]

    pid_summary.append({"product_id": product_id, "summary": summary})

# Set summary property of Product nodes
summary_query = """
UNWIND $summaries AS summary
MATCH (p:Product {product_id: summary.product_id})
SET p.summary = summary.summary
"""
graphdb.query(summary_query, summaries=pid_summary)

([], True)

### Use Cases

**Workflow**:

*Use Case Nodes*: Create nodes in the KG that represent common use cases for products. These nodes connect user intents to specific product interactions.

*Example*: A query for “insomnia” could pull up a use case node linked to “Epsom salt baths for sleep,” with a description of how and why the product fits the use case. Queries can be mapped to use-cases via semantic search (query -> generated use case -> semantic search on db of use-cases). Other examples of intent: “office chair for back pain,” “eco-friendly cleaning supplies”

*Narrative Explanation*: Tie each use case node to a narrative or scenario-based explanation that illustrates how the product solves the problem. For example, “Many users find that taking an Epsom salt bath before bed helps them relax and prepare for sleep.”

In [375]:
class UseCase(BaseModel):
    title: str = Field(
        title="title",
        description="A short title for the use case.",
        examples=["Relaxation via aromatherapy", "Sleep aid", "Skin care"],
        min_length=5,
    )
    explanation: str = Field(
        title="explanation",
        description="A brief explanation of why this subcategory is suitable for this use case.",
        examples=[
            "It is commonly used in diffusers to promote relaxation and reduce stress due to its natural calming properties.",
            "It is known to promote sleep, making it an effective natural remedy for people with insomnia or sleep disorders when diffused at night.",
            "It has antimicrobial properties, making it useful as a topical treatment for minor skin irritations and acne.",
        ],
        min_length=10,
    )


class UseCaseList(BaseModel):
    use_cases: List[UseCase] = Field(
        title="use_cases",
        description="A list of common use cases and explanations for the product.",
        min_length=2,
        max_length=10,
    )

    def __len__(self):
        return len(self.use_cases)

    def __getitem__(self, item):
        return self.use_cases[item]

    def __iter__(self):
        return iter(self.use_cases)
    
# capture everything between ```json and ```)
json_pattern = re.compile(r'(?<=```json)(.*?)(?=```)', re.DOTALL)

def usecase_parser(message: AIMessage) -> List[Dict[str, str]]:
    usecases = []
    # find first match
    match = json_pattern.search(message.content)
    if match:
        usecases_str = match.group(0).strip()
        try:
            usecases = literal_eval(usecases_str)
        except ValueError:
            try:
                usecases = json.loads(usecases_str)
            except json.JSONDecodeError:
                pass
    return usecases

prompt_text = """
Generate a list of use-cases for the following subcategory of products, given the subcategory title and the main category that it belongs to on an eCommerce website. 
Each use-case should include a **title** and a brief **explanation** of why it is suitable for that use-case. 

**Expected Output Format:**
```json
[
    {{
      "title": "Relaxation via aromatherapy,
      "explanation": "It is commonly used in diffusers to promote relaxation and reduce stress due to its natural calming properties."
    }},
    {{
      "title": "Sleep aid",
      "explanation": "It is known to promote sleep, making it an effective natural remedy for people with insomnia or sleep disorders when diffused at night."
    }},
    {{
      "title": "Skincare",
      "explanation": "It has antimicrobial properties, making it useful as a topical treatment for minor skin irritations and acne."
    }},
    ...
]
```
The goal is to provide a comprehensive list of common scenarios where the subcategory of products is used, along with a succinct explanation of why this use case exists.
It is recommended to generate at least 5 use-cases. Do not generate less than 2 or exceed 10 use-cases.

Subcategory: {subcategory}
Main Category: {main_category}
""".strip()

prompt_template = PromptTemplate(
    template=prompt_text,
    input_variables=["subcategory", "main_category"],
    input_types={"subcategory": str, "main_category": str},
)
# usecase_llm = llm.with_structured_output(UseCaseList)
usecase_chain = prompt_template | llm | usecase_parser

In [376]:
def run_example(example=None):
    set_debug(True)
    if not example:
        row = products_df.sample().iloc[0]
        categories = literal_eval(row["categories"])
        subcategory = random.choice(categories)
        example = {"main_category": row["main_category"], "subcategory": subcategory}
    
    result = usecase_chain.invoke(input=example)
    set_debug(False)
    return result

In [ ]:
result = run_example()

In [ ]:
result

In [ ]:
subcategories = set()
for index, row in products_df.iterrows():
    for subcategory in row["categories"]:
        subcategories.add((row["main_category"], subcategory))
subcategories = list(subcategories)
len(subcategories)

In [ ]:
if os.path.exists("data/use_cases.json"):
    with open("data/use_cases.json") as f:
        use_cases = json.load(f)
else:
    use_cases = []

errors = []

for main_category, subcategory in tqdm(subcategories):
    try:
        result = usecase_chain.invoke(
            {"subcategory": subcategory, "main_category": main_category}
        )
        for usecase in result:
            title = usecase["title"]
            explanation = usecase["explanation"]
            use_cases.append(
                {
                    "title": title,
                    "explanation": explanation,
                    "subcategory": subcategory,
                    "main_category": main_category,
                }
            )
    except Exception as e:
        errors.append(index)
        print(f"Error processing product {index}")
        with open("data/errors_use_cases.txt", "w") as f:
            f.write("\n".join(errors))
    else:
        with open("data/use_cases.json", "w") as f:
            json.dump(use_cases, f, indent=2)

In [35]:
### error resolution...

with open("data/use_cases.json", "r") as f:
    use_cases = json.load(f)

print(len({entry['title'] for entry in use_cases}))
use_cases, unique_uc = reduce_keys(use_cases, key_field='title', similarity_threshold=90)
print(len({entry['title'] for entry in use_cases}))

1847
1641


In [38]:
# delete existing use-case nodes
delete_query = """
MATCH (u:UseCase)
CALL (u) {
DETACH DELETE u
} IN TRANSACTIONS OF 10000 ROWS;
"""
graphdb.query(delete_query)

create_usecases_query = """
UNWIND $data AS data
// Match the existing Category node
MATCH (category:Category {name: data.main_category})
// Match the existing Subcategory node
MATCH (subcategory:Subcategory {name: data.subcategory})
// Create the UseCase node
MERGE (usecase:UseCase {title: data.title})
// Create the USED_FOR relationship with the explanation property
MERGE (usecase)-[:USED_FOR {explanation: data.explanation}]->(subcategory)
"""
batch_size = 1000
for i in trange(0, len(use_cases), batch_size):
    batch = use_cases[i:i+batch_size]
    graphdb.query(create_usecases_query, data=batch)

  0%|          | 0/3 [00:00<?, ?it/s]